<a href="https://colab.research.google.com/github/SamPheeney/PCOL3911_Sam-Pheeney/blob/main/GetData_GenerateFingerPrints(1024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This time we can get the values for the organophosphates. I copied the supplementary table into a CSV and put it on Github.

#BEFORE YOU START:

Save your own copy of this apge in your drive:

Click **File** and choose **"save in drive"**

In [1]:
#Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
#Load data from Github
data = pd.read_csv("https://raw.githubusercontent.com/SamPheeney/PCOL3911_Sam-Pheeney/main/PCOL3911%20-%20Sam%20Pheeney/gsar_a_1049665_sm4518.csv")


In [3]:
data.head(2)

,ID,SMILES,Expr,DCW,Calc,Expr-Calc,Set,DCW.1,Calc.1,Expr-Calc.1,Set.1,DCW.2,Calc.2,Expr-Calc.2,Set.2
0,1,CSP(=O)(c1ccccc1)c1ccccc1,2.34,39.01370,2.5431,-0.2031,Tr,30.28552,2.6612,-0.3212,ST,34.79287,2.3165,0.0235,Ca
1,2,CCSP(=O)(c1ccccc1)c1ccccc1,2.69,40.13724,2.8413,-0.1513,Va,31.29530,2.9406,-0.2506,Va,36.00961,2.6870,0.0030,Va


In [4]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 39.8 MB/s eta 0:00:00


In [5]:
#Add a molecule column and make sure RDkt can convert all SMILES
from rdkit import Chem, DataStructs
from rdkit.Chem import PandasTools, AllChem
PandasTools.AddMoleculeColumnToFrame(data,'SMILES','Molecule')
data[["SMILES","Molecule"]].head(1)

,SMILES,Molecule
0,CSP(=O)(c1ccccc1)c1ccccc1,<rdkit.Chem.rdchem.Mol object at 0x7befc4e18ac0>


In [6]:
#Check for smiles that rdkit can't convert to molecule. If sum = 0 then they are all OK
data.Molecule.isna().sum()

0

In [7]:
#Define function to generate fp's from SMILES
#Here we are producing a Morgan FP with radius 2 and calling for 1024 bits

from rdkit.Chem import rdFingerprintGenerator

mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=1024)

mfp_list = []
for mol in data['Molecule']:
  mfp = mfpgen.GetFingerprintAsNumPy(mol)
  mfp_list.append(mfp)

data['MFP'] = mfp_list

In [8]:
data.head(2)

,ID,SMILES,Expr,DCW,Calc,Expr-Calc,Set,DCW.1,Calc.1,Expr-Calc.1,Set.1,DCW.2,Calc.2,Expr-Calc.2,Set.2,Molecule,MFP
0,1,CSP(=O)(c1ccccc1)c1ccccc1,2.34,39.01370,2.5431,-0.2031,Tr,30.28552,2.6612,-0.3212,ST,34.79287,2.3165,0.0235,Ca,<rdkit.Chem.rdchem.Mol object at 0x7befc4e18ac0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,CCSP(=O)(c1ccccc1)c1ccccc1,2.69,40.13724,2.8413,-0.1513,Va,31.29530,2.9406,-0.2506,Va,36.00961,2.6870,0.0030,Va,<rdkit.Chem.rdchem.Mol object at 0x7befc4e18b30>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
data_fp = data["MFP"].apply(pd.Series)

In [10]:
data_fp.head(2)

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#Bring back the outcome column
data_fp.insert(1024, "Expr", data["Expr"])
data_fp.head(3)

,0,1,2,3,4,5,6,7,8,9,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,Expr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.34
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.69
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.08


In [12]:
#Save dataframe as csv
data_fp.to_csv('organophosphate_fp.csv',index=None)

## Now you can download this new csv file and store it on your Github site for modelling.
Click on the folder icon on the left and find the file inside the explorer window (on the left) and choose download.